In [35]:
# First cell - Imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from datetime import datetime
import json
import os
from dotenv import load_dotenv

In [36]:
# Second Cell - Data Extraction/Load
load_dotenv()

# Get API key from environment
API_KEY = os.getenv('FRED_API_KEY')
BASE_URL = 'https://api.stlouisfed.org/fred'

def get_fred_series(series_id, observation_start=None):
    """
    Fetch data series from FRED API
    
    Parameters:
    series_id (str): FRED series identifier
    observation_start (str): Start date in YYYY-MM-DD format
    
    Returns:
    pandas.DataFrame: Time series data
    """
    # Build API URL
    url = f"{BASE_URL}/series/observations"
    
    params = {
        'series_id': series_id,
        'api_key': API_KEY,
        'file_type': 'json',
        'observation_start': observation_start if observation_start else '1976-01-01'
    }
    
    # Make API request
    response = requests.get(url, params=params)
    data = response.json()
    
    # Convert to DataFrame
    df = pd.DataFrame(data['observations'])
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    
    return df.set_index('date')['value'] #set date index so pandas automatically knows the date (i.e. resampling)

# Get our series
yield_spread = get_fred_series('T10Y2Y')
gdp = get_fred_series('GDPC1')
fed_funds = get_fred_series('DFF')
unemployment = get_fred_series('UNRATE')
option_adjusted_spread = get_fred_series('BAMLH0A0HYM2')
delinquency_rate = get_fred_series('DRCCLACBS')

# Create main dataframe
df = pd.DataFrame({
    'yield_spread': yield_spread,
    'gdp': gdp,
    'fed_funds':fed_funds, 
    'unemployment': unemployment,
    'option_adjusted_spread': option_adjusted_spread,
    'delinquency_rate': delinquency_rate
})


In [37]:
# Third Cell - Data Transformation

# Calculate GDP growth rate (annualized)
df['gdp_growth'] = df['gdp'].pct_change(periods=4) * 100

df['delinquency_rate_clean'] = df['delinquency_rate'].ffill()

# Get end of month value for option adjusted spread
monthly_spreads = (df
    .resample('M')['option_adjusted_spread']
    .last()  # Takes last value of each month
    .reset_index()
    .rename(columns={'option_adjusted_spread': 'monthly_spread'})
)

# Fill down (forward fill)
df = df.merge(monthly_spreads, on='date', how='left')
df['monthly_spread'] = df['monthly_spread'].ffill()
df.to_excel('stats.xlsx')

C:\Users\mrasm\AppData\Local\Temp\ipykernel_28256\115380690.py:4: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['gdp_growth'] = df['gdp'].pct_change(periods=4) * 100
C:\Users\mrasm\AppData\Local\Temp\ipykernel_28256\115380690.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample('M')['option_adjusted_spread']


In [21]:
# Fourth Cell - Data Visualization


            yield_spread       gdp  fed_funds  unemployment  \
date                                                          
1976-01-01           NaN  6323.649       5.37           7.9   
1976-01-02           NaN       NaN       5.28           NaN   
1976-01-03           NaN       NaN       5.28           NaN   
1976-01-04           NaN       NaN       5.28           NaN   
1976-01-05           NaN       NaN       5.29           NaN   
...                  ...       ...        ...           ...   
2024-11-04          0.14       NaN       4.83           NaN   
2024-11-05          0.07       NaN       4.83           NaN   
2024-11-06          0.15       NaN       4.83           NaN   
2024-11-07          0.10       NaN       4.83           NaN   
2024-11-08          0.04       NaN        NaN           NaN   

            option_adjusted_spread  delinquency_rate  gdp_growth  \
date                                                               
1976-01-01                     NaN          